<a href="https://colab.research.google.com/github/drinith/analises_smartbugs/blob/main/Verificacao_saida_smartbugs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Verificar a contagem diferente que dá nos resultados do smartbugs**
https://github.com/smartbugs/smartbugs-results


In [9]:
from google.colab import drive
import pandas as pd
import matplotlib.pyplot as plt
import os
import json


drive.mount('/content/drive')

#link root
link_dennis = './drive/MyDrive/Colab Notebooks/Artigo/tables_pesquisa/mythril'

link_smartbugs = './drive/MyDrive/Colab Notebooks/Artigo/smartbugs_results/mythril'
class SmartContract:

  _nome=''
  _vulnerabilidades=[]

  def __init__(self,nome):

      self._nome = nome
      self._vulnerabilidades=[]

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Função que busca as saídas do mythril, com as vulnerabilidades, e o transforma em um dataframe colocando inclusive a repetição da mesma vulnerabilidade se encontrada no smartcontract




In [10]:
def montar_dataframe_todo_erro() -> pd.DataFrame:

  #Carregando os nomes da pasta
  folder_list_mythril = os.listdir(link_smartbugs+'/curated')
  print(folder_list_mythril)

  #Lista que vai conter os smart contracts levantados
  lista_smart_contracts = []

  for pasta in  folder_list_mythril:
    json_arquivo = open(link_smartbugs+'/curated/'+pasta+'/result.json')
    #print(json_arquivo)
    smart_json = json.load(json_arquivo)
    #print(smart_json)

    #Instanciando smart contracts com o nome levantado do json
    sc = SmartContract(smart_json['contract'])

    try:
      vulnerabilidades_lista = smart_json['analysis']['issues']
    except Exception as e:

      print('A exceção foi ',e)


    for vulnerabilidade in vulnerabilidades_lista:
      sc._vulnerabilidades.append(vulnerabilidade['title'])

    #preenchendo a lista com as informações
    lista_smart_contracts.append(sc)

  #print(lista_smart_contracts)

  print(lista_smart_contracts)
  df = pd.DataFrame(columns=lista_smart_contracts)

  #df = pd.DataFrame(columns=[scs._vulnerabilidades for scs in lista_smart_contracts])
  df = pd.DataFrame()
  df['name']=1

  #Tentando criar as colunas com os nomes das vulnerabililidades
  for scs in lista_smart_contracts:

    df[scs._vulnerabilidades]=1

  index=0
  df = pd.DataFrame([[0]* len(df.columns)]*len(lista_smart_contracts),columns=df.columns)

  for scs in lista_smart_contracts:


    df.loc[index,'name']=scs._nome

    for coluna in scs._vulnerabilidades:

      df.loc[index,coluna]+=1
    index+=1

  return df

In [11]:
df = montar_dataframe_todo_erro()

df

['dos_simple', 'BECToken', 'etheraffle', 'arbitrary_location_write_simple', 'dos_number', 'dos_address', 'ERC20', 'auction', 'blackjack', 'crypto_roulette', 'etherpot_lotto', 'guess_the_random_number', 'ether_lotto', 'FibonacciBalance', 'etherstore', 'FindThisHash', 'eth_tx_order_dependence_minimal', 'governmental_survey', 'incorrect_constructor_name1', 'incorrect_constructor_name2', 'incorrect_constructor_name3', 'integer_overflow_benign_1', 'integer_overflow_mapping_sym_1', 'integer_overflow_multitx_multifunc_feasible', 'integer_overflow_1', 'king_of_the_ether_throne', 'integer_overflow_add', 'integer_overflow_mul', 'integer_overflow_multitx_onefunc_feasible', 'integer_overflow_minimal', 'lottery', 'mycontract', 'name_registrar', 'modifier_reentrancy', 'multiowned_vulnerable', 'lotto', 'list_dos', 'odds_and_evens', 'mapping_write', 'lucky_doubler', 'reentrance', 'proxy', 'overflow_simple_add', 'old_blockhash', 'random_number_generator', 'parity_wallet_bug_1', 'parity_wallet_bug_2', '

,name,Integer Overflow,Transaction order dependence,Dependence on predictable environment variable,Multiple Calls,Integer Underflow,Exception state,Ether send,Unchecked CALL return value,Call data forwarded with delegatecall(),DELEGATECALL to a user-supplied address,Message call to external contract,State change after external call,Use of tx.origin,Dependence on predictable variable,Unchecked SUICIDE
0,dos_simple,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,BECToken,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,etheraffle,4,3,1,2,0,0,0,0,0,0,0,0,0,0,0
3,arbitrary_location_write_simple,3,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,dos_number,3,0,0,0,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,timelock,3,0,1,0,0,0,0,0,0,0,0,0,0,0,0
65,unprotected0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
66,tokensalechallenge,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
67,unchecked_return_value,0,0,0,0,0,0,0,1,0,0,2,0,0,0,0


Criando o dicionário para tradução apartir da tabela https://github.com/smartbugs/smartbugs/wiki/Vulnerabilities-mapping


In [12]:

#Criando uma tabela zerada que tenha como nome das colunas a classificação DASP
df_dasp= pd.DataFrame(0, index=range(df.shape[0]),columns=['access_control','arithmetic','denial_service','reentrancy','unchecked_low_calls','bad_randomness','front_running',	'time_manipulation',	'short_addresses',	'Other',	'Ignore'])


#Setando o 'name' como index da tabela que veio do mythril
df.set_index('name', inplace=True)

df_dasp.index = df.index


#Discionário da tabela https://github.com/smartbugs/smartbugs/wiki/Vulnerabilities-mapping
dasp_dic = {'Call data forwarded with delegatecall()':'access_control','DELEGATECALL to a user-supplied address':'access_control','Ether send':'access_control',
            'Unchecked SUICIDE':'access_control','Use of tx.origin':'access_control',	'Integer Overflow ':'arithmetic','Integer Underflow':'arithmetic',	'Message call to external contract':'reentrancy','State change after external call':'reentrancy',
            'Unchecked CALL return value':'unchecked_low_calls','Transaction order dependence':'front_running',	'Dependence on predictable environment variable':'Other',	'Dependence on predictable variable':'Other',
            'Exception state':'Other','Multiple Calls':'Ignore'}




#Colocando os encontros das vulnerabilidade de acordo com a tradução para o DASP
for column in df.columns:

  df_dasp[dasp_dic[column]]= df_dasp[dasp_dic[column]] | df[column]


#Somando os encontros de uma vulnerabilidade na tablea
df_dasp_soma = df_dasp.sum().to_frame()

#Retirando o index
df_dasp_soma.reset_index(inplace=True)

#Renomeando as colunas
df_dasp_soma.columns=['Vulnerabildiade','Quantidade']

df_dasp_soma



,Vulnerabildiade,Quantidade
0,access_control,22
1,arithmetic,92
2,denial_service,0
3,reentrancy,13
4,unchecked_low_calls,30
5,bad_randomness,0
6,front_running,21
7,time_manipulation,0
8,short_addresses,0
9,Other,30
